# Mask R-CNN
Mask R-CNN [ ICCV 2017  ·  Kaiming He, Georgia Gkioxari, Piotr Dollár, Ross Girshick ]

https://paperswithcode.com/paper/mask-r-cnn

### [ Abstract ]
***

- Faster R-CNN에 Mask branch 구조를 추가하여 object detection 모델을 instance segmentation 모델로 전환한 것
- 각 Faster R-CNN의 RPN(Region propsal Network)에서 얻은 RoI(Region of Interest)에 FCN을 적용하여 segmentation을 구현한 것이라고 이해할 수 있다.

### [ Faster R-CNN ]
***

<img src="./Mask R-CNN7.png" width=700>

<img src="./Mask R-CNN1.png" width=450>

- Region Proposal Network(RPN)은 Image를 입력으로 받아서 객체에 대한 Score와 함께 사각형의 물체 Proposal을 추출한다. Fully Convolution으로 구성되어 있으며, 이 이유는 Fast R-CNN과 계산을 공유하기 위함이다.

- Region Proposal을 생성하기 위해 feature map을 입력으로 받아 사용하는 Convolution Network를 구성하고, Sliding Window를 통해 Region of Interest (RoI)

- 이후 Fully Connected Layer를 통해 Box-regression과 Box-classification layer를 생성한다.

### [ Anchors ]
***

<img src="./Mask R-CNN2.png" width=800>

- sliding window의 각 위치마다 여러 개의 Region Proposal을 만들게 된다. k는 각 위치마다 생성되는 Proposal의 최대 갯수이다. 따라서 4k coordinates는 k개의 박스의 좌표이므로 4k이고, 2k scores는 k개의 object의 score 값으로 물체인지 아닌지 판단하므로 2k개이다. 

- 여기서 k개의 Box를 앵커의 개념을 도입한다. k개의 Proposal은 k개의 reference box에 대하여 parameterize될 수 있다. Anchor는 해당 sliding window의 중앙에 위치한다. 이 논문은 3개의 Scale과 3개의 Ratio를 고려하여 총 9개의 Anchor를 사용하였다.

- Translation-Invariant Anchors
    - 본 논문에서 중요한 Property가 바로 translation invariant이다. CNN을 공부하면 다 알고 있겠지만 translation invariant는 대상의 위치에 영향을 받지 않고, 위치 변화에 무관하다는 뜻이다. 앵커의 관점과 Proposal을 계산하는 관점에서 모두 translation invariant하다. 물체의 위치가 바뀌면 바뀐 물체의 영역을 뽑아낼 수 있는 것이다. 또한 이런 특성은 모델의 크기를 줄일 수 있다. MultiBox Method는 (4 + 1) x 800-dimensional FC Output Layer를 사용한다. 하지만 Anchor를 사용하게 되면 (4 + 2) x 9-dimensional Convolution Output Layer를 사용하게 되면서 Parameter 수가 감소하게 된다. 그렇기 때문에 Overfitting의 위험도 감소하게 된다. 

- Multi-Scale Anchors as Regression References
    - 저자가 설계한 Anchor는 여러 크기와 비율의 객체를 잘 찾는다. 기존의 여러 크기의 객체를 탐지하는 방법은 아래 사진과 같이 크게 두 가지가 존재하였다.

<img src="./Mask R-CNN3.png" width=800>

- 첫 번째는 Feature Pyramid 방식이다. 영상을 Resize하여 여러 크기의 영상을 만들고, 각 영상에서의 Feature를 뽑아내어 사용한다. 이는 유용하게 사용되지만 시간이 오래 걸린다는 단점이 있다. 

- 두 번째 방법은 여러 크기의 Sliding Window를 쓰는 것인데, 여러 크기의 Convolution을 사용한다고 생각하면 된다. 이 방식은 첫 번째 방법과 함께 사용되는 것이 일반적이다. 

- 이와 비교하여 이 논문에서 말하는 방법은 pyramid of anchor 방식이다. 계산량도 적고 사진도 한 장에 대해서만 계산하면 된다.

### [ Architecture ]
***

<img src="./Mask R-CNN4.png" width=600>

Mask R-CNN은 Faster R-CNN의 RPN에서 얻은 RoI(Region of Interest)에 대하여 객체의 class를 예측하는 classification branch, bbox regression을 수행하는 bbox regression branch와 평행으로 segmentation mask를 예측하는 mask branch를 추가한 구조를 가지고 있습니다. mask branch는 각각의 RoI에 작은 크기의 FCN(Fully Convolutional Network)가 추가된 형태입니다. segmentation task를 보다 효과적으로 수행하기 위해 논문에서는 객체의 spatial location을 보존하는 RoIAlign layer를 추가했습니다. 

<img src="./Mask R-CNN5.png" width=500>

- RoIPooling:
    - RoIPool은 각 RoI에서 small feature map을 추출하기 위한 표준 연산
    - RoIPool : 다른 사이즈의 Region Proposal이 들어와도, max pooling을 이용하여 output size를 동일하게 만듦 (Faster R-CNN에서 나오는 개념)
    - RoIPool은 RoI를 feature map으로 quantization하게 되는 데, 이 과정에서 RoI와 추출된 feature 사이에 오정렬을 초래하며 픽셀 단위로 예측하는 mask에 큰 악영향을 끼침
    - 이 단점을 해결하기 위해 추출한 feature를 input에 적절하게 정렬하는 RoIAlign layer를 제안했다.


<img src="./Mask R-CNN6.png" width=600>

- RoIAlign:
    - bilinear interpolation 연산을 사용하여 각 RoI bin의 샘플링된 4개의 위치에서 input feature의 정확한 값을 계산
    - 그 후 결과를 max 또는 평균낸다.
    - Faster R-CNN의 quantization은 사용되지 않음